Setup environment

In [ ]:
python3 -m venv venv # setup environment
source venv/bin/activate # activate environment
pip install django # install django framework
pip install djangorestframework # install django rest framework

Create a new project name `ebooksapi`

In [ ]:
django-admin startproject ebooksapi

Create new app name `ebooks`

In [ ]:
python manage.py startapp ebooks

Add the app name into the `INSTALLED_APPS` array in `ebooksapi/settings.py` file

In [ ]:
# Application definition

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'ebooks'
]

Create new classes in `ebooks/models.py` file


In [ ]:
from django.db import models
from django.core.validators import *
# Create your models here.
class Ebook(models.Model):
    title = models.CharField(max_length=200)
    author = models.CharField(max_length=200)
    description = models.TextField()
    publication_date = models.DateField()
    created_date = models.DateTimeField(auto_now_add=True)
    updated_date = models.DateTimeField(auto_now=True)
    
    def __str__(self) -> str:
        return f"{self.title} - {self.author}"
    
class Review(models.Model):
    created_date = models.DateTimeField(auto_now_add=True)
    updated_date = models.DateTimeField(auto_now=True)
    review_author = models.CharField(max_length=10, blank=True, null=True)
    review = models.TextField(blank=True, null=True)
    rating = models.PositiveIntegerField(validators=[MinValueValidator(1), MaxValueValidator(5)])
    ebook = models.ForeignKey(Ebook, on_delete=models.CASCADE, related_name="reviews")
    
    
    def __str__(self) -> str:
        return f"{self.rating}"

Run the commands to make migrations in database

In [ ]:
python manage.py makemigrations
python manage.py migrate

Run command to create super user and setup `superuser account` in `admin page`
Command will create a `admin.py` file  in `news` folder

In [ ]:
python manage.py createsuperuser # command will create a `admin.py` file  in `news` folder

In `news/admin.py` add following code to register our models to `admin page`

In [ ]:
from newsapi.news import models

# Register your models here.
admin.site.register(models.Review)
admin.site.register(models.Ebook)

Create new folder `api` in `ebooks`, and new file `serializers.py`. Add these content to the file

In [ ]:
from rest_framework import serializers
from ebooks.models import Ebook, Review

class ReviewSerializer(serializers.ModelSerializer):
    class Meta:
        model = Review
        fields = "__all__"
        
        
class EbookSerializer(serializers.ModelSerializer):
    reviews = ReviewSerializer(many=True, read_only=True)
    class Meta:
        model = Ebook
        fields = "__all__"

 [25] <h2> `GenericAPIView && Mixins`</h2>

Create `views.py` and adding this code:

In [ ]:
from rest_framework import generics, mixins

from ebooks.models import Ebook
from ebooks.api.serializers import EbookSerializer


class EbookListCreateApiView(mixins.ListModelMixin,
                            mixins.CreateModelMixin, 
                            generics.GenericAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    
    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)
    
    def post(self, request, *args, **kwargs):
        return self.create(request, *args, **kwargs)


class EbookDetailApiView(mixins.RetrieveModelMixin,
                            mixins.UpdateModelMixin, 
                            mixins.DestroyModelMixin,
                            generics.GenericAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    
    def get(self, request, *args, **kwargs):
        return self.retrieve(request, *args, **kwargs)
    
    def put(self, request, *args, **kwargs):
        return self.update(request, *args, **kwargs)
    
    def delete(self, request, *args, **kwargs):
        return self.destroy(request, *args, **kwargs)


Create new file `urls.py` in `api` folder

In [ ]:
from django.urls import path
from .views import EbookListCreateApiView, EbookDetailApiView


urlpatterns = [
    path("ebooks/", EbookListCreateApiView.as_view(), name="ebook-list"),
    path("ebooks/<int:pk>", EbookDetailApiView.as_view(), name="ebook-detail")
]


Add new line `path("api/", include("ebooks.api.urls"))` into `urlpatterns` of `ebooksapi/ebooksapi/urls.py`, then it will become:

In [ ]:
urlpatterns = [
    path('admin/', admin.site.urls),
    path("api/", include("ebooks.api.urls"))
]

Let check the result in `http://127.0.0.1:8000/api/ebooks`

[26] <h2>`Concrete View Class`</h2>
replace code in `views.py` with the below:

In [ ]:
from rest_framework import generics, mixins

from ebooks.models import Ebook
from ebooks.api.serializers import EbookSerializer


class EbookListCreateApiView(generics.ListCreateAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    
    
class EbookDetailApiView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    

The api will still function the same.
Now we add some more code to handle `Review`. <br>
In `views.py` add:

In [ ]:
 
class ReviewListCreateApiView(generics.ListCreateAPIView):
    queryset = Review.objects.all()
    serializer_class = ReviewSerializer
    
    def perform_create(self, serializer):
        ebook_pk = self.kwargs.get("ebook_pk")
        ebook = generics.get_object_or_404(Ebook, pk=ebook_pk)
        serializer.save(ebook=ebook)
        
    
class ReviewDetailApiView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Review.objects.all()
    serializer_class = ReviewSerializer

Update `ReviewSerializer` class, exclude `ebook` field to manually handle the param.

In [ ]:
class ReviewSerializer(serializers.ModelSerializer):
    class Meta:
        model = Review
        # fields = "__all__"
        exclude = ("ebook",)

In `api/urls.py` we add:

In [ ]:
from django.urls import path
from .views import EbookListCreateApiView, EbookDetailApiView, ReviewListCreateApiView, ReviewDetailApiView


urlpatterns = [
    path("ebooks/", EbookListCreateApiView.as_view(), name="ebook-list"),
    path("ebooks/<int:pk>", EbookDetailApiView.as_view(), name="ebook-detail"),
    path("ebooks/<int:ebook_pk>/reviews/", ReviewListCreateApiView.as_view(), name="ebook-review"),
    path("reviews/<int:pk>", ReviewDetailApiView.as_view(), name="review-detail"),
]


Now in browser, let's try to list the `Review` by url GET `http://127.0.0.1:8000/api/ebooks/1/reviews/` or Create new review by POST `http://127.0.0.1:8000/api/ebooks/1/reviews/` the same url <b> <h2>`Permission System`</h2> [1]

Restrict all API must be authenticated by adding below dict in `settings.py`

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticated',
    ]
}

Access `/api/ebooks/` on browser will give you the `403` error: 
`{
    "detail" : "Authentication credentials were not provided"
}`
<br>
<br>
Replace the `rest_framework.permissions.IsAuthenticated` with `rest_framework.permissions.IsAuthenticatedOrReadOnly` 

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.IsAuthenticatedOrReadOnly',
    ]
}

This setting is every useful but it has limitation and lacking flexibility. We will define our own class to handle permission ourself.<br>
Create the file `permissions.py` in `api/` folder with content:

In [ ]:
from rest_framework import permissions

class IsAdminUserOrReadOnly(permissions.IsAdminUser):
    def has_permission(self, request, view):
        is_admin = super().has_permission(request, view) # check if the current request was sent by an admin?
        
        return request.method in permissions.SAFE_METHODS or is_admin # allow access if the method is safe to access, or the user is an admin`

`Create a standard user to limit the privilege of a user (for demostration)`

Modify the file `api/views.py`

In [ ]:
...
from ebooks.api.serializers import EbookSerializer, ReviewSerializer
from ebooks.api.permissions import IsAdminUserOrReadOnly

class EbookListCreateApiView(generics.ListCreateAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    permission_classes = [IsAdminUserOrReadOnly]
    
    
class EbookDetailApiView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    permission_classes = [IsAdminUserOrReadOnly]
    
...

Logout the `admin` user and login again by new standard user just created.

[2] Continuing the discession we have started in the previous lesson, we are now going to see how to secure our `review` instances so that they can be updated or deleted only by the same users who have created them. <br> In order to do so we will first need to modify our `Review` model, binding it to Django's User model using a ForeignKey field.

In `models.py` we add `from django.contrib.auth.models import User`. We also change the type of `review_author` property

In [ ]:
from django.contrib.auth.models import User
...
class Review(models.Model):
    review_author = models.ForeignKey(User, on_delete=models.CASCADE) #
...

In [ ]:
(venv) VNBDEPMLTP416:ebooksapi sungpham$ python manage.py makemigrations
It is impossible to change a nullable field 'review_author' on review to non-nullable without providing a default. This is because the database needs something to populate existing rows.
Please select a fix:
 1) Provide a one-off default now (will be set on all existing rows with a null value for this column)
 2) Ignore for now. Existing rows that contain NULL values will have to be handled manually, for example with a RunPython or RunSQL operation.
 3) Quit and manually define a default value in models.py.
Select an option: __

Enter `1`

Modify all the conflictions...<br>
Add to the `ReviewSerializer` class:

In [ ]:
...
class ReviewSerializer(serializers.ModelSerializer):
    review_author = serializers.StringRelatedField(read_only=True)
    
...

In `api/views.py`, modify the class `ReviewCreatedAPIView` 

In [ ]:
from rest_framework.validators import ValidationError

class ReviewCreateApiView(generics.CreateAPIView):
    queryset = Review.objects.all()
    serializer_class = ReviewSerializer
    permission_classes = [permissions.IsAuthenticatedOrReadOnly]
    
    def perform_create(self, serializer):
        ebook_pk = self.kwargs.get("ebook_pk")
        ebook = generics.get_object_or_404(Ebook, pk=ebook_pk)
        review_author = self.request.user # set the author of the review is the current user
        
        review_queryset = Review.objects.filter(ebook=ebook, 
                                                review_author=review_author) # filter the review that belongs to the current user
        if review_queryset.exists(): # if the current user has reviewed this book, then raise the exception
            raise ValidationError("You have already review this book!")
        serializer.save(ebook=ebook, review_author = review_author)
        

Now go to url on browser `/api/ebooks/2/reviews/`.

In `permissions.py`, create a new permission class, to check if current user is the author of the review or not?

In [ ]:
class IsReviewAuthorOrReadOnly(permissions.BasePermission):
    def has_object_permission(self, request, view, obj):
        if request.method in permissions.SAFE_METHODS:
            return True
        
        return obj.review_author == request.user

In `api/views.py`, add the permission class above to prevent other user to `DELETE` or `UPDATE` the review:

In [ ]:
    
class ReviewDetailApiView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Review.objects.all()
    serializer_class = ReviewSerializer
    permission_classes = [IsReviewAuthorOrReadOnly]

[29]<h2>`Pagination`</h2>
For a quick set up we just need to add to the `settings.py`

In [ ]:
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.LimitOffsetPagination',
    'PAGE_SIZE': 3
}

In real world scenario, we might want to create a new class to better controlling the pagination. Let's comment out the content:

In [ ]:
# REST_FRAMEWORK = {
#     'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.LimitOffsetPagination',
#     'PAGE_SIZE': 3
# }

Create `paginations.py` in `api` with content:

In [ ]:
from rest_framework.pagination import PageNumberPagination

class SmallSetPagination(PageNumberPagination):
    page_size = 3

Add `SmallSetPagination` into usage in `api/views.py`.

In [ ]:
from ebooks.api.paginations import SmallSetPagination

class EbookListCreateApiView(generics.ListCreateAPIView):
    queryset = Ebook.objects.all()
    serializer_class = EbookSerializer
    permission_classes = [IsAdminUserOrReadOnly]
    pagination_class = SmallSetPagination